In [499]:
import pandas as pd
import sklearn
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from pymorphy2 import MorphAnalyzer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB

# from catboost import CatBoostClassifier

import common

In [391]:
morph = MorphAnalyzer()

In [500]:
stemmer = SnowballStemmer("russian")

def stem(string):
    
    tokens = [stemmer.stem(token) for token in common.word_tokenize(string)]

    return " ".join(tokens)

In [502]:
def first_form(string):
    
    tokens = common.word_tokenize(string)
    
    for i in range(len(tokens)):
        if tokens[i].islower():
            parse = morph.parse(tokens[i])
            tokens[i] = parse[0].normal_form
    
    return " ".join(tokens)

In [503]:
synonyms = {
        "зарплата": ["зп"],
        "электричка": ["элка"]
    }

replace_map = {}

for key, value in synonyms.items():
    for syn in value:
        replace_map[first_form(syn)] = first_form(key)

def replace_synonyms(string):
    
    tokens = common.word_tokenize(string)
    
    for i in range(len(tokens)):
        tokens[i] = replace_map.get(tokens[i], tokens[i])
    
    
    return " ".join(tokens)

In [504]:
russian_stopwords = stopwords.words("russian")

def delete_stopwords(string):
    
    tokens = common.word_tokenize(string)
    
    for token in tokens:
        if token in russian_stopwords:
            tokens.remove(token)
    
    return " ".join(tokens)

In [405]:
def print_conf_matrix(y_test, y_predicted, categories):
    
    ma = sklearn.metrics.confusion_matrix(y_test, y_predicted).astype(str)
    ma = np.insert(ma, 0, categories, axis=1)
    ma = np.insert(ma, 0, [""] + categories, axis=0)
    
    for row in range(ma.shape[0]):
        for col in range(ma.shape[1]):
            if col != row and ma[row, col] != "0" and col > 0 and row > 0:
                print("{0:<10}".format("<" + ma[row, col] + ">"), end="")
            else:
                print("{0:<10}".format(ma[row, col]), end="")
        print()

In [406]:
def print_scores(y_test, y_predicted, categories):
    
    print("Accuracy: {0}".format(sklearn.metrics.accuracy_score(y_test, y_predicted)), end="\n\n")
    
    print(" " * 15, end="")
    for cat in categories:
        print("{0:<10}".format(cat), end="")
    
    print("\nPrecision:     ", end="")
    for digit in sklearn.metrics.precision_score(y_test, y_predicted, average=None):
        print("{0:<10}".format(round(digit, 3)), end="")
    
    print("\nRecall:        ", end="")
    for digit in sklearn.metrics.recall_score(y_test, y_predicted, average=None):
        print("{0:<10}".format(round(digit, 3)), end="")
        
    print("\nF1:            ", end="")
    for digit in sklearn.metrics.f1_score(y_test, y_predicted, average=None):
        print("{0:<10}".format(round(digit, 3)), end="")

In [444]:
data = pd.read_csv("./messages/30_09_2017_19_37.csv", delimiter=";")

In [445]:
data.loc[:, "message"] = data["message"].apply(str.lower).apply(replace_letterdigits)
data.loc[:, "message"] = data["message"].apply(delete_punctuation)

In [446]:
data["timepointer"] = data["message"].apply(replace_timepointer)

In [447]:
data["datepointer"] = data["timepointer"].apply(replace_datepointer)

In [448]:
data["weightpointer"] = data["datepointer"].apply(replace_weightpointer)

In [449]:
data["no_num"] = data["weightpointer"].apply(replace_numbers)

In [450]:
data["no_stopwords"] = data["no_num"].apply(delete_stopwords)

In [451]:
data["first_form"] = data["no_stopwords"].apply(first_form)

In [452]:
data["synonyms"] = data["first_form"].apply(replace_synonyms)

In [462]:
print(data.shape)
data.head(10)

(266, 9)


,message,category,timepointer,datepointer,weightpointer,no_num,no_stopwords,first_form,synonyms
0,23800 доплаты за отпуска в крок,income,23800 доплаты за отпуска в крок,23800 доплаты за отпуска в крок,23800 доплаты за отпуска в крок,NUM доплаты за отпуска в крок,NUM доплаты отпуска крок,NUM доплата отпуск крок,NUM доплата отпуск крок
1,54782.52 зарплата,income,54782.52 зарплата,54782.52 зарплата,54782.52 зарплата,NUM зарплата,NUM зарплата,NUM зарплата,NUM зарплата
2,6086.5 зарплата,income,6086.5 зарплата,6086.5 зарплата,6086.5 зарплата,NUM зарплата,NUM зарплата,NUM зарплата,NUM зарплата
3,1500 кошулька дала немного денег,income,1500 кошулька дала немного денег,1500 кошулька дала немного денег,1500 кошулька дала немного денег,NUM кошулька дала немного денег,NUM кошулька дала немного денег,NUM кошулька дать немного деньга,NUM кошулька дать немного деньга
4,зп 36000,income,зп 36000,зп 36000,зп 36000,зп NUM,зп NUM,зп NUM,зарплата NUM
5,9 сентября олег дал 30000,income,9 сентября олег дал 30000,DATEPOINTER олег дал 30000,DATEPOINTER олег дал 30000,DATEPOINTER олег дал NUM,DATEPOINTER олег дал NUM,DATEPOINTER олег дать NUM,DATEPOINTER олег дать NUM
6,ещё зп 4000,income,ещё зп 4000,ещё зп 4000,ещё зп 4000,ещё зп NUM,ещё зп NUM,ещё зп NUM,ещё зарплата NUM
7,вчера ходил в бассейн проплыл 600 метров,other,вчера ходил в бассейн проплыл 600 метров,DATEPOINTER ходил в бассейн проплыл 600 метров,DATEPOINTER ходил в бассейн проплыл 600 метров,DATEPOINTER ходил в бассейн проплыл NUM метров,DATEPOINTER ходил бассейн проплыл NUM метров,DATEPOINTER ходить бассейн проплыть NUM метр,DATEPOINTER ходить бассейн проплыть NUM метр
8,вес вчера 74.7,weight,вес вчера 74.7,вес DATEPOINTER 74.7,WEIGHTPOINTER DATEPOINTER,WEIGHTPOINTER DATEPOINTER,WEIGHTPOINTER DATEPOINTER,WEIGHTPOINTER DATEPOINTER,WEIGHTPOINTER DATEPOINTER
9,сегодня читаю три товарища,event,сегодня читаю три товарища,DATEPOINTER читаю три товарища,DATEPOINTER читаю три товарища,DATEPOINTER читаю три товарища,DATEPOINTER читаю товарища,DATEPOINTER читать товарищ,DATEPOINTER читать товарищ


In [476]:
X = data["synonyms"]
Y = data["category"]

In [477]:
count_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfTransformer()

X_count = count_vectorizer.fit_transform(X)
X_tfidf = tfidf_vectorizer.fit_transform(X_count)
X_tfidf.shape

(266, 159)

In [481]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X_tfidf, Y, test_size=0.3, random_state=16)

In [482]:
X_train_text, X_test_text, *_ = sklearn.model_selection.train_test_split(X, Y, test_size=0.3, random_state=16)

#### SVM

In [483]:
svm = sklearn.linear_model.SGDClassifier(loss="hinge", alpha=1e-3, max_iter=5)
svm.fit(X_train, Y_train)

predicted = svm.predict(X_test)

In [484]:
print_scores(Y_test, predicted, bayes.classes_.tolist())
print("\n\n\nConfusion matrix:\n")
print_conf_matrix(Y_test, predicted, bayes.classes_.tolist())

Accuracy: 0.9375

               event     income    other     outcome   timespend weight    
Precision:     0.5       1.0       0.0       0.944     1.0       1.0       
Recall:        0.5       0.25      0.0       1.0       1.0       1.0       
F1:            0.5       0.4       0.0       0.971     1.0       1.0       


Confusion matrix:

          event     income    other     outcome   timespend weight    
event     1         0         0         <1>       0         0         
income    0         1         0         <3>       0         0         
other     <1>       0         0         0         0         0         
outcome   0         0         0         67        0         0         
timespend 0         0         0         0         5         0         
weight    0         0         0         0         0         1         


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


##### Что именно не так классифицировалось

In [498]:
results = pd.DataFrame()

results["text"] = X_test_text
results["y_test"] = Y_test
results["predicted"] = predicted

results[~(results.y_test == results.predicted)]

,text,y_test,predicted
7,DATEPOINTER ходить бассейн проплыть NUM метр,other,event
0,NUM доплата отпуск крок,income,outcome
10,сделать зарядка глаз,event,outcome
3,NUM кошулька дать немного деньга,income,outcome
5,DATEPOINTER олег дать NUM,income,outcome


## Классификация без использования text-based фичей

In [507]:
data_two = pd.DataFrame()
data_two["message"] = data["synonyms"]
data_two["category"] = data["category"]

data_two.head()

,message,category
0,NUM доплата отпуск крок,income
1,NUM зарплата,income
2,NUM зарплата,income
3,NUM кошулька дать немного деньга,income
4,зарплата NUM,income


In [ ]:
data_two["has_hum"] = data_two["message"].str

In [514]:
data_two.message

0                           NUM доплата отпуск крок
1                                      NUM зарплата
2                                      NUM зарплата
3                  NUM кошулька дать немного деньга
4                                      зарплата NUM
5                         DATEPOINTER олег дать NUM
6                                  ещё зарплата NUM
7      DATEPOINTER ходить бассейн проплыть NUM метр
8                         WEIGHTPOINTER DATEPOINTER
9                        DATEPOINTER читать товарищ
10                             сделать зарядка глаз
11                                    WEIGHTPOINTER
12                      DATEPOINTER сходить бассейн
13                                    WEIGHTPOINTER
14               пробежать DATEPOINTER NUM километр
15                                    WEIGHTPOINTER
16                           DATEPOINTER в спортзал
17                            купить NUM доллар NUM
18                                    WEIGHTPOINTER
19          

#### CatBoost - неуместно использовать с некатегориальными фичами, нужно применить для другого способа классификации

In [107]:
# X_train_array = X_train.toarray()
# X_test_array = X_test.toarray()

In [108]:
# le = sklearn.preprocessing.LabelEncoder()
# le.fit(Y_test.append(Y_train))

In [109]:
# Y_train_digits = le.transform(Y_train)

In [110]:
# catboost = CatBoostClassifier(iterations=2, depth=2, learning_rate=1, loss_function='Logloss', verbose=True)
# catboost.fit(X_train_array, Y_train_digits, verbose=True, cat_features=None)

# predicted = catboost.predict(X_test_array)